In [ ]:
import numpy as np

# Conversion Binaire

In [ ]:
def dataToBinaryArray(data,nbit) :
    if isinstance(data,str) :
        binaire_separee = np.array([format(ord(car),"0"+str(nbit)+"b") for car in data])
    else :
        binaire_separee = np.array([format(car,"0"+str(nbit)+"b") for car in data])
    binaire = np.array([int(num) for i in range(len(binaire_separee)) for num in binaire_separee[i]])
    return binaire,nbit

In [ ]:
def binaryArrayToDataInteger(binary_array,nbit) :
    arr = np.array_split(binary_array,len(binary_array)//nbit)
    return [int("".join(map(str,arr[i])),2) for i in range(len(arr))]

In [ ]:
def binaryArrayToDataString(binary_array,nbit) :
    arr = np.array_split(binary_array,len(binary_array)//nbit)
    return "".join([chr(int("".join(map(str,arr[i])),2)) for i in range(len(arr))])

# Type de Lecture

In [ ]:
def lectureLigne(arr) :
    return (arr.reshape(-1),arr.shape)

In [ ]:
def dlectureLigne(vecteur,dim) :
    return np.array(np.split(vecteur,len(vecteur) // dim[1]))


In [ ]:
def lectureColonne(arr) :
    return (np.hstack(np.column_stack(arr)),arr.shape)


In [ ]:
def dlectureColonne(vecteur,dim) :
    return np.transpose(np.vstack(np.split(vecteur,len(vecteur) // dim[0])))

In [ ]:
def lectureZigzag(arr) :
    arr_flip = np.fliplr(arr)
    if arr.shape[1] % 2 == 0 :
        zig = np.hstack([np.diag(arr_flip,k=i) if i % 2 == 0 else np.flip(np.diag(arr_flip,k=i)) for i in range(arr.shape[1] - 1, -arr.shape[0], -1)])
    elif arr.shape[1] % 2 != 0 :
        zig = np.hstack([np.diag(arr_flip,k=i) if i % 2 != 0 else np.flip(np.diag(arr_flip,k=i)) for i in range(arr.shape[1] - 1, -arr.shape[0], -1)])
    return (zig,arr.shape)

In [ ]:
def dlectureZigzag(zigzag,dim) :
    arr = np.zeros(dim,dtype=np.uint8)
    indices = np.cumsum([len(np.diag(arr,i)) for i in range(arr.shape[1]-1,-arr.shape[0],-1) ])
    zigzag_split = np.split(zigzag,indices)[:-1]
    zigzag_split = [zigzag_split[i] if i % 2 != 0 else np.flip(zigzag_split[i]) for i in range(len(zigzag_split))]
    for j in range(arr.shape[1]) :
        i = 0
        tab = zigzag_split[j]
        while i < arr.shape[0] and j > - 1 :
            arr[i,j] = tab[i]
            i += 1
            j -= 1
    for i in range(1,arr.shape[0]) :
        j = arr.shape[1] - 1
        tab = zigzag_split[j+i]
        k = 0
        while i < arr.shape[0] and j > - 1 and k < len(tab) :
            arr[i,j] = tab[k]
            i += 1
            j -= 1
            k += 1
    return arr

# RLE

In [ ]:
def rlec(sequence) :
    val = sequence[0]
    codage = [] + [0] if val==0 else [] 
    l = 0
    for bit in sequence :
        l += 1
        if bit != val :
            codage.append(l-1)
            val = bit
            l = 1
        if l > 255 :
            l -= 1
            codage.append(l)
            codage.append(0)
            l -= 254
    if l != 0 : 
        codage.append(l)
    return codage

In [ ]:
def rled(sequence) :
    val = False if sequence[0]==0 else True
    sequence = sequence[1:] if val==False else sequence
    data = []
    for i in range(len(sequence)) :
        data.extend(np.repeat(int(val),sequence[i]))
        val = not val
    return data

# HUFFMAN

In [ ]:
def symbole_frequence(data) :
    tab = list(data)
    dictt = {elem if isinstance(elem,str) else chr(elem) : tab.count(elem) for elem in set(tab)}
    return dict(sorted(dictt.items(),key=lambda item : item[1]))

In [ ]:
def huffmanC(dictio) :
    liste_symb = list(dictio.items())
    codage_dict = {car :"" for car,freq in liste_symb}
    while len(liste_symb) > 1 :
        min1 = min(liste_symb,key=lambda item:item[1])
        liste_symb.pop(liste_symb.index(min1))
        min2 = min(liste_symb,key=lambda item:item[1])
        liste_symb.pop(liste_symb.index(min2))
        for car in min1[0] :
            codage_dict[car] += "0" 
        for car in min2[0] :
            codage_dict[car] += "1"
        elem = [min1[0]+min2[0],min1[1]+min2[1]]
        liste_symb.append(elem)
    return {car  : code[::-1] for car,code in list(codage_dict.items())}

In [ ]:
def sequenceBinaireHuffmanCodage(data) :
    dictt = symbole_frequence(data)
    dict_code = huffmanC(dictt)
    return [dict_code[car] if isinstance(car,str) else dict_code[chr(car)] for car in data]

# LZ78

In [ ]:
def lz78c(sequence) :
    sequence = list(sequence)
    dictt = {"":0}
    etiquette = []
    courant = ""
    for car in sequence :
        courant += car
        if courant not in dictt.keys() :
            dictt[courant] = len(dictt.keys())
            etiquette.append((dictt[courant[:-1]],car))
            courant = ""
    if courant != "" :                  
        etiquette.append((dictt[courant],""))
    return etiquette


In [ ]:
def lz78d(etiquette) :
    dictt = {0:""}
    for etq in etiquette :
        dictt[len(dictt)] = dictt[etq[0]] + etq[1]
    dictt.pop(0)
    return dictt.values()

lz78d(lz78c(map(str,[1,2,3,4,5])))

# LZW

In [ ]:
def lzwc(sequence,dict_codage) :
    courant = sequence[0] 
    codage = []
    for i in range(1,len(sequence)) :
        suivant = sequence[i]
        if courant + suivant in dict_codage :
            courant += suivant
        else :
            dict_codage[courant+suivant] = len(dict_codage) 
            codage.append(dict_codage[courant])
            courant = suivant
    codage.append(dict_codage[courant])
    return codage


In [ ]:
def lzwd(codage,dict_codage) :
    dict_codage = {v: k for k, v in dict_codage.items()}
    anc = dict_codage[codage[0]]
    decodage = [] + [anc]
    for i in range(1,len(codage)) :
        nv = codage[i]
        dict_codage[len(dict_codage)] = anc + dict_codage[nv][0] if nv in dict_codage else anc + anc
        decodage.append(dict_codage[nv])
        anc = dict_codage[nv]
    return decodage

# Codage en ligne

## RZ

In [ ]:
def RZUni(sequence,maxV) :
    new_sequence = np.array(sequence).reshape(-1,1)
    signal = np.where(new_sequence==[1],[maxV,0],[0])
    return signal

In [ ]:
def RZBi(sequence,maxV=5,minV=-5) :
    new_sequence = np.array(sequence).reshape(-1,1)
    signal = np.where(new_sequence==[1],[maxV,0],[minV,0])
    return signal

In [ ]:
def RZDecodage(signal) :
    arr = np.column_stack(np.where(signal==[5,0],1,0))[0]
    return arr

## NRZ NRZI

In [ ]:
def NRZ(sequence,maxV,minV) :
    new_sequence = np.array(sequence).reshape(-1,1)
    signal = np.where(new_sequence==1,[maxV,maxV],[minV,minV])
    return signal
   

In [ ]:
def NRZI(sequence,maxV,minV):
    code = []
    code.append([maxV,maxV] if sequence[0]==1 else [minV,minV])
    for i in range(1,len(sequence)) :
        code.append(code[i-1] if sequence[i]==1 else [-code[i-1][0],-code[i-1][1]])
    return code

# MANCHESTER MANCHESTER DIFF

In [ ]:
def manchester(sequence ,maxV,minV) :
    new_sequence = np.array(sequence).reshape(-1,1)
    signal = np.where(new_sequence==1,[maxV,minV],[minV,maxV])
    return signal

In [ ]:
def manchesterDiff(sequence,maxV,minV):
    code = []
    code.append([maxV,-maxV] if sequence[0]==1 else [-minV,minV])
    for i in range(1,len(sequence)) :
        code.append([code[i-1][1],code[i-1][0]] if sequence[i]==1 else code[i-1])
    return code

In [ ]:
def manchesterDecodage(signal,maxV,minV):
    seq = np.where(signal==[5,-5],1,0)[:,0]
    return seq

In [ ]:
def manchesterDiffDecodage(signal):
    seq = [] + [1] if signal[0]==[5,-5] else [] + [0]
    for i in range(1,len(signal)) :
        seq.append(1 if signal[i]!=signal[i-1] else 0)
    return seq

## MILLER

In [ ]:
def miller(sequence) :
    code = [] + [[5,-5]] if sequence[0]==1 else [] + [[-5,-5]]
    for i in range(1,len(sequence)) :
        if sequence[i]==0 and sequence[i-1] == 0 :
            code.append([-code[-1][1],-code[-1][1]])
        elif sequence[i] == 0 and sequence[i-1]==1 :
            code.append([code[-1][1],code[-1][1]])
        else :
            code.append([code[-1][1],-code[-1][1]])
    return code

## CMI

In [ ]:
def CMI(sequence) :
    code = [] + [[-5,5]] if sequence[0]==1 else [] + [[-5,-5]]
    for i in range(1,len(sequence)) :
        if sequence[i]==0 and sequence[i-1] == 0 :
            code.append([-code[-1][1],-code[-1][1]])
        elif sequence[i] == 0 and sequence[i-1]==1 :
            code.append([code[-1][1],code[-1][1]])
        else :
            code.extend([[-5,5]])
    return code

## AMI

In [ ]:
def AMI(sequence) :
    change = 5 
    code = []
    for i in range(len(sequence)) :
        if sequence[i] == 1 :
            code.append(change)
            change = - change
        else : code.append(0)
    return code

## MTL3

In [ ]:
def MLT3(sequence) :
    code = [0] if sequence[0] == 0 else [5]
    transition = [5,0,-5,0] if code[0] == 0 else [0,-5,0,5]
    for i in range(1,len(sequence)) :
        if sequence[i] == 1 :
            code.append(transition[0])
            transition = transition[1:] + [transition[0]]
        else : code.append(code[-1])
    return code

## 2BIQ

In [ ]:
def c2BIQ(sequence) :
    dictt = {(0,0): -3,(0,1):-1,(1,1):1,(1,0):3}
    sequence = np.array(sequence).reshape(-1,1)
    sequence = np.concatenate((sequence[:-1:2],sequence[1::2]),axis=1)
    code = [dictt[tuple(seq)] for seq in sequence]
    return code